In [6]:
import numpy as np
import os
import pandas as pd
from scipy import integrate

### This airPLS code is used for baseline removal of SERS spectra.

This code is written by its original author, Yizeng Liang and Zhimin Zhang. We did not make any modifications and simply applied it to our SERS data.

Reference:
Z.-M. Zhang, S. Chen, and Y.-Z. Liang, Baseline correction using adaptive iteratively reweighted penalized least squares. Analyst 135 (5), 1138-1146 (2010).

In [7]:
'''
airPLS.py Copyright 2014 Renato Lombardo - renato.lombardo@unipa.it
Baseline correction using adaptive iteratively reweighted penalized least squares

This program is a translation in python of the R source code of airPLS version 2.0
by Yizeng Liang and Zhang Zhimin - https://code.google.com/p/airpls
Reference:
Z.-M. Zhang, S. Chen, and Y.-Z. Liang, Baseline correction using adaptive iteratively reweighted penalized least squares. Analyst 135 (5), 1138-1146 (2010).

Description from the original documentation:

Baseline drift always blurs or even swamps signals and deteriorates analytical results, particularly in multivariate analysis.  It is necessary to correct baseline drift to perform further data analysis. Simple or modified polynomial fitting has been found to be effective in some extent. However, this method requires user intervention and prone to variability especially in low signal-to-noise ratio environments. The proposed adaptive iteratively reweighted Penalized Least Squares (airPLS) algorithm doesn't require any user intervention and prior information, such as detected peaks. It iteratively changes weights of sum squares errors (SSE) between the fitted baseline and original signals, and the weights of SSE are obtained adaptively using between previously fitted baseline and original signals. This baseline estimator is general, fast and flexible in fitting baseline.


LICENCE
This program is free software: you can redistribute it and/or modify
it under the terms of the GNU Lesser General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU Lesser General Public License for more details.

You should have received a copy of the GNU Lesser General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>
'''

import numpy as np
from scipy.sparse import csc_matrix, eye, diags
from scipy.sparse.linalg import spsolve

def WhittakerSmooth(x,w,lambda_,differences=1):
    '''
    Penalized least squares algorithm for background fitting
    
    input
        x: input data (i.e. chromatogram of spectrum)
        w: binary masks (value of the mask is zero if a point belongs to peaks and one otherwise)
        lambda_: parameter that can be adjusted by user. The larger lambda is,  the smoother the resulting background
        differences: integer indicating the order of the difference of penalties
    
    output
        the fitted background vector
    '''
    X=np.matrix(x)
    m=X.size
    E=eye(m,format='csc')
    for i in range(differences):
        E=E[1:]-E[:-1] # numpy.diff() does not work with sparse matrix. This is a workaround.
    W=diags(w,0,shape=(m,m))
    A=csc_matrix(W+(lambda_*E.T*E))
    B=csc_matrix(W*X.T)
    background=spsolve(A,B)
    return np.array(background)

def airPLS(x, lambda_=100, porder=1, itermax=15):
    '''
    Adaptive iteratively reweighted penalized least squares for baseline fitting
    
    input
        x: input data (i.e. chromatogram of spectrum)
        lambda_: parameter that can be adjusted by user. The larger lambda is,  the smoother the resulting background, z
        porder: adaptive iteratively reweighted penalized least squares for baseline fitting
    
    output
        the fitted background vector
    '''
    m=x.shape[0]
    w=np.ones(m)
    for i in range(1,itermax+1):
        z=WhittakerSmooth(x,w,lambda_, porder)
        d=x-z
        dssn=np.abs(d[d<0].sum())
        if(dssn<0.001*(abs(x)).sum() or i==itermax):
            if(i==itermax): print('WARING max iteration reached!')
            break
        w[d>=0]=0 # d>0 means that this point is part of a peak, so its weight is set to 0 in order to ignore it
        w[d<0]=np.exp(i*np.abs(d[d<0])/dssn)
        w[0]=np.exp(i*(d[d<0]).max()/dssn) 
        w[-1]=w[0]
    return z



#### Batch processing using airPLS

The airPLS code is applied to all csvs in the given root folder, and will save a processed csv for each original csv in the output folder.

In [ ]:
def area_normalization(x, y):
    '''
    Area normalization, ensure the area under the normalized spectra = 1
    
    input
        x: wavenumber column
        y: intensity column
    
    output
        the normalized intensity column
    '''
    if np.any(np.isnan(x)) or np.any(np.isnan(y)) or np.any(np.isinf(x)) or np.any(np.isinf(y)):
        print("Warning: NaN or inf values detected in the data")
        return y
    area = integrate.trapz(y, x)
    if area != 0:
        return y / area
    else:
        print("Warning: Area is zero, returning original spectrum")
        return y

In [9]:
# Path to the folder containing the CSV files
root_folder_path = r"E:\Yanjun Yang\Virus in Saliva - Single detection (Original intensity data)"
output_folder_path = r"E:\Yanjun Yang\airPLS removed"

# Process each CSV file in the folder
for file_name in os.listdir(root_folder_path):
    if file_name.endswith('.csv'):  # Check if the file is a CSV
        print("Processing " + file_name)
        file_path = os.path.join(root_folder_path, file_name)
        df = pd.read_csv(file_path, sep='\t')
        x = df.iloc[:, 0].values

        # Area normalization, ensure the area under the normalized spectra = 1
        for column in df.columns[1:]:
            df[column] = area_normalization(x, df[column].values)
        
        # Apply airPLS to every column except the first one (wavenumbers)
        for col in df.columns[1:]:
            baseline = airPLS(df[col].values, lambda_=90, porder=1, itermax=100)
            df[col] = df[col].values - baseline
        
        # Do area normalization again to ensure that each output spectrum has the same scale
        for column in df.columns[1:]:
            df[column] = area_normalization(x, df[column].values)
        
        # Define the new file name
        new_file_name = file_name.replace('.csv', '_airPLS_processed.csv')
        new_file_path = os.path.join(output_folder_path, new_file_name)
        
        # Save the processed dataframe to a new CSV file
        df.to_csv(new_file_path, sep='\t', index=False)


Processing CoVNL63-100000_FluB-100.csv
Processing CoVNL63-100000_FluB-100000.csv
Processing CoVNL63-100000_FluB-12500.csv
Processing CoVNL63-100000_FluB-1562.csv
Processing CoVNL63-100000_FluB-195.csv
Processing CoVNL63-100000_FluB-25000.csv
Processing CoVNL63-100000_FluB-3125.csv
Processing CoVNL63-100000_FluB-391.csv
Processing CoVNL63-100000_FluB-50.csv
Processing CoVNL63-100000_FluB-50000.csv
Processing CoVNL63-100000_FluB-6250.csv
Processing CoVNL63-100000_FluB-781.csv
Processing CoVNL63-100_FluB-100.csv
Processing CoVNL63-100_FluB-100000.csv
Processing CoVNL63-100_FluB-12500.csv
Processing CoVNL63-100_FluB-1562.csv
Processing CoVNL63-100_FluB-195.csv
Processing CoVNL63-100_FluB-25000.csv
Processing CoVNL63-100_FluB-3125.csv
Processing CoVNL63-100_FluB-391.csv
Processing CoVNL63-100_FluB-50.csv
Processing CoVNL63-100_FluB-50000.csv
Processing CoVNL63-100_FluB-6250.csv
Processing CoVNL63-100_FluB-781.csv
Processing CoVNL63-12500_FluB-100.csv
Processing CoVNL63-12500_FluB-100000.csv